# Miami Traffic Incidents Exploratory Notebook

In [2]:
# Load dependencies
import pandas as pd
import requests
import json
from google.cloud import bigquery
from google.oauth2 import service_account
import getpass
import pytz

In [3]:
# Make the request to grab the data
url = "https://traffic.mdpd.com/api/traffic"
resp = requests.get(url)

In [4]:
# Display raw results
print(type(resp.text))
resp.text

<class 'str'>


'[{"CreateTime":"2025-07-18T21:20:03","Signal":"TRAFFIC ACCIDENT","Address":"NW 7TH ST / NW 72ND AVE","Location":"NB","Grid":"1242","MapX":null,"MapY":null,"Longitude":-80.31245791,"Latitude":25.77716579},{"CreateTime":"2025-07-18T21:10:20","Signal":"HIT AND RUN JUST OCCURRED","Address":"SW 264TH ST / US 1","Location":"ALL INFO VIA SPANISH INTERPRETER","Grid":"5487","MapX":null,"MapY":null,"Longitude":-80.4246234,"Latitude":25.52167043},{"CreateTime":"2025-07-18T21:07:30","Signal":"TRAFFIC ACCIDENT","Address":"NW 22ND AVE / NW 54TH ST","Location":"","Grid":"8968","MapX":null,"MapY":null,"Longitude":-80.23269176,"Latitude":25.82417772},{"CreateTime":"2025-07-18T21:03:30","Signal":"TRAFFIC ACCIDENT","Address":"NW 91ST ST / NW 8TH AVE","Location":"","Grid":"0741","MapX":null,"MapY":null,"Longitude":-80.21159559,"Latitude":25.85803868},{"CreateTime":"2025-07-18T21:03:30","Signal":"TRAFFIC ACCIDENT","Address":"NW 91ST ST / NW 8TH AVE","Location":"","Grid":"0741","MapX":null,"MapY":null,"Lon

In [5]:
# Parse the string inside the JSON response (converts from string to a list of dictionaries)
incidents = json.loads(resp.text)

# Explore the data
print(type(incidents))
print(incidents[0])

<class 'list'>
{'CreateTime': '2025-07-18T21:20:03', 'Signal': 'TRAFFIC ACCIDENT', 'Address': 'NW 7TH ST / NW 72ND AVE', 'Location': 'NB', 'Grid': '1242', 'MapX': None, 'MapY': None, 'Longitude': -80.31245791, 'Latitude': 25.77716579}


In [6]:
# Check how many dictionaries in the list
len(incidents)

14

In [7]:
# Inspect a few records
incidents[0:2]

[{'CreateTime': '2025-07-18T21:20:03',
  'Signal': 'TRAFFIC ACCIDENT',
  'Address': 'NW 7TH ST / NW 72ND AVE',
  'Location': 'NB',
  'Grid': '1242',
  'MapX': None,
  'MapY': None,
  'Longitude': -80.31245791,
  'Latitude': 25.77716579},
 {'CreateTime': '2025-07-18T21:10:20',
  'Signal': 'HIT AND RUN JUST OCCURRED',
  'Address': 'SW 264TH ST / US 1',
  'Location': 'ALL INFO VIA SPANISH INTERPRETER',
  'Grid': '5487',
  'MapX': None,
  'MapY': None,
  'Longitude': -80.4246234,
  'Latitude': 25.52167043}]

In [8]:
# Convert list of dictionaries into a dataframe
df = pd.DataFrame(incidents)
df.head()

,CreateTime,Signal,Address,Location,Grid,MapX,MapY,Longitude,Latitude
0,2025-07-18T21:20:03,TRAFFIC ACCIDENT,NW 7TH ST / NW 72ND AVE,NB,1242,None,None,-80.312458,25.777166
1,2025-07-18T21:10:20,HIT AND RUN JUST OCCURRED,SW 264TH ST / US 1,ALL INFO VIA SPANISH INTERPRETER,5487,None,None,-80.424623,25.521670
2,2025-07-18T21:07:30,TRAFFIC ACCIDENT,NW 22ND AVE / NW 54TH ST,,8968,None,None,-80.232692,25.824178
3,2025-07-18T21:03:30,TRAFFIC ACCIDENT,NW 91ST ST / NW 8TH AVE,,0741,None,None,-80.211596,25.858039
4,2025-07-18T21:03:30,TRAFFIC ACCIDENT,NW 91ST ST / NW 8TH AVE,,0741,None,None,-80.211596,25.858039


While observing the traffic website, I have noticed duplicate rows on several occasions. It's not clear if there is a reason for the duplicate rows but I will remove them. May need to revisit this.

In [9]:
# Remove duplicates
df = df.drop_duplicates()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 0 to 13
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CreateTime  13 non-null     object 
 1   Signal      13 non-null     object 
 2   Address     13 non-null     object 
 3   Location    13 non-null     object 
 4   Grid        13 non-null     object 
 5   MapX        0 non-null      object 
 6   MapY        0 non-null      object 
 7   Longitude   13 non-null     float64
 8   Latitude    13 non-null     float64
dtypes: float64(2), object(7)
memory usage: 1.0+ KB
None


,CreateTime,Signal,Address,Location,Grid,MapX,MapY,Longitude,Latitude
0,2025-07-18T21:20:03,TRAFFIC ACCIDENT,NW 7TH ST / NW 72ND AVE,NB,1242,None,None,-80.312458,25.777166
1,2025-07-18T21:10:20,HIT AND RUN JUST OCCURRED,SW 264TH ST / US 1,ALL INFO VIA SPANISH INTERPRETER,5487,None,None,-80.424623,25.521670
2,2025-07-18T21:07:30,TRAFFIC ACCIDENT,NW 22ND AVE / NW 54TH ST,,8968,None,None,-80.232692,25.824178
3,2025-07-18T21:03:30,TRAFFIC ACCIDENT,NW 91ST ST / NW 8TH AVE,,0741,None,None,-80.211596,25.858039
5,2025-07-18T20:55:26,TRAFFIC ACCIDENT WITH INJURIES,1606 SW 3RD CT,LAT: <25.466233> LONG: <-80.497780>,2633,None,None,-80.497780,25.466233


In [10]:

# Convert to Eastern Time, then to UTC
df['CreateTime'] = pd.to_datetime(df['CreateTime']).dt.tz_localize('US/Eastern')
df['CreateTime'] = df['CreateTime'].dt.tz_convert('UTC')

# Convert Signal to categorical datatype
df['Signal'] = df['Signal'].astype('category')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 0 to 13
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   CreateTime  13 non-null     datetime64[ns, UTC]
 1   Signal      13 non-null     category           
 2   Address     13 non-null     object             
 3   Location    13 non-null     object             
 4   Grid        13 non-null     object             
 5   MapX        0 non-null      object             
 6   MapY        0 non-null      object             
 7   Longitude   13 non-null     float64            
 8   Latitude    13 non-null     float64            
dtypes: category(1), datetime64[ns, UTC](1), float64(2), object(5)
memory usage: 1.1+ KB


In [12]:
# Connect to BigQuery
# Enter Project ID
project_id = input("Enter your GCP project ID: ") # miami-traffic

# Enter the path to JSON key file
key_path = getpass.getpass("Enter full path to your service account JSON key file: ")

# Authenticate
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=project_id)

# List datasets to test connection
print("🔄 Connecting to BigQuery...")
datasets = list(client.list_datasets())
if datasets:
    print("✅ Connection successful! Found the following datasets:")
    for d in datasets:
        print(f"  - {d.dataset_id}")
else:
    print("✅ Connected, but no datasets found in this project.")

🔄 Connecting to BigQuery...
✅ Connection successful! Found the following datasets:
  - mdpd_traffic_data


In [13]:
# Location to upload data (project->dataset->table name)
table_id = "miami-traffic.mdpd_traffic_data.mdpd_data"

# Append new data to table
job = client.load_table_from_dataframe(df, table_id)
job.result()  # indicates job is complete
print("✅ Data uploaded to BigQuery!")

✅ Data uploaded to BigQuery!
